# Evaluating the models on Test set

In [4]:
from GIST import *

## Our tests will include evaluating the models (MLP and RNN) on GIST and raw features

### Create the gist features for test set

In [5]:
from pathlib import Path
import os
import glob
import pandas as pd
import cv2
import pandas as pd
import os
root = "dataset"
output_dir = "gist_test_features/"
Path(root+"/"+output_dir).mkdir(parents=True, exist_ok=True)
dir_path = root+"/test/*.png"
files = glob.glob(dir_path)
for file in files:
    print("Processing:",file)
    img = cv2.imread(file,0)
    gist_feature =  gist(img,image_size=28)
    file_path = root + "/"+output_dir+os.path.basename(file)
    np.save(file_path,gist_feature)

    

Processing: dataset/test/img_019.png
Processing: dataset/test/img_000.png
Processing: dataset/test/img_001.png
Processing: dataset/test/img_002.png
Processing: dataset/test/img_003.png
Processing: dataset/test/img_004.png
Processing: dataset/test/img_005.png
Processing: dataset/test/img_006.png
Processing: dataset/test/img_007.png
Processing: dataset/test/img_008.png
Processing: dataset/test/img_009.png
Processing: dataset/test/img_010.png
Processing: dataset/test/img_011.png
Processing: dataset/test/img_012.png
Processing: dataset/test/img_013.png
Processing: dataset/test/img_014.png
Processing: dataset/test/img_015.png
Processing: dataset/test/img_016.png
Processing: dataset/test/img_017.png
Processing: dataset/test/img_018.png
Processing: dataset/test/img_020.png
Processing: dataset/test/img_021.png
Processing: dataset/test/img_022.png
Processing: dataset/test/img_023.png
Processing: dataset/test/img_024.png
Processing: dataset/test/img_025.png
Processing: dataset/test/img_026.png
P

### Set the device

In [7]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)


Using PyTorch version: 1.12.0  Device: cuda


In [8]:
from nn_model import MLP,RNN
model = torch.load("models/gist_1d_320_vector_mlp.net")
model.eval()

MLP(
  (fc1): Linear(in_features=320, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=4, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

### Lets now evaluate the test set as GIST features for MLP model

In [9]:
from custom_dataset import GistTestDataSet,TestDataSet
batch_size = 40
dataset = GistTestDataSet("dataset/gist_test_features/*.npy") 
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
input_size = 320
sequence_length = 1

for paths,data in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data.to(device))
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)   

pred

tensor([1, 3, 2, 3, 1, 3, 0, 2, 1, 1, 0, 2, 0, 1, 3, 1, 0, 3, 0, 3, 1, 3, 1, 2,
        2, 3, 0, 3, 3, 2, 0, 0, 3, 3, 0, 3, 0, 1, 3, 1], device='cuda:0')

### Lets save the predictions

In [10]:
labels_df = pd.DataFrame(columns=["file","label"])
for index in range(0,len(paths)):
    labels_df.loc[-1] = [os.path.basename(paths[index]),pred[index].cpu().numpy()]
    labels_df.index = labels_df.index + 1  # shifting index
    labels_df = labels_df.sort_index()
labels_df.to_csv("gist_mlp_test_lables.csv")

### Lets now evaluate the test set as raw features for MLP model


In [11]:
model = torch.load("models/raw_1d_784_vector_mlp.net")
model.eval()

MLP(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=4, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [12]:
from torchvision import transforms
batch_size = 40
transform = transforms.Compose([transforms.ToTensor(),                                     
        transforms.Lambda(lambda x: torch.flatten(x))])
dataset = TestDataSet("dataset/test/*.png",transform)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
input_size = 784
sequence_length = 1

for paths,data in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data.to(device))
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)   

pred    

tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], device='cuda:0')

### Lets save the predictions

In [13]:
labels_df = pd.DataFrame(columns=["file","label"])
for index in range(0,len(paths)):
    labels_df.loc[-1] = [os.path.basename(paths[index]),pred[index].cpu().numpy()]
    labels_df.index = labels_df.index + 1  # shifting index
    labels_df = labels_df.sort_index()
labels_df.to_csv("raw_mlp_test_lables.csv")

### Lets now evaluate the test set as GIST features for RNN model

In [14]:
model = torch.load("models/gist_1d_320_vector_rnn.net")
model.eval()

RNN(
  (rnn): RNN(320, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)

In [15]:
batch_size = 40
dataset = GistTestDataSet("dataset/gist_test_features/*.npy") 
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
input_size = 320
sequence_length = 1

for paths,data in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data.unsqueeze(1).to(device))
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)   

pred

tensor([1, 0, 2, 3, 1, 3, 0, 2, 2, 1, 0, 2, 0, 0, 3, 1, 0, 3, 0, 1, 1, 3, 0, 2,
        2, 3, 0, 3, 1, 2, 0, 0, 3, 1, 0, 3, 0, 1, 3, 1], device='cuda:0')

### Lets save the predictions

In [14]:
labels_df = pd.DataFrame(columns=["file","label"])
for index in range(0,len(paths)):
    labels_df.loc[-1] = [os.path.basename(paths[index]),pred[index].cpu().numpy()]
    labels_df.index = labels_df.index + 1  # shifting index
    labels_df = labels_df.sort_index()
labels_df.to_csv("gist_rnn_test_lables.csv")

### Lets now evaluate the test set as raw features for RNN model

In [17]:
model = torch.load("models/raw_28x28_image_rnn.net")
model.eval()

RNN(
  (rnn): RNN(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=3584, out_features=4, bias=True)
)

In [18]:

batch_size = 40
transform = transforms.Compose([transforms.ToTensor()])
dataset = TestDataSet("dataset/test/*.png",transform)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
input_size = 28
sequence_length = 28

for paths,data in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data.squeeze(1).to(device))
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)   

pred    

tensor([0, 0, 3, 2, 0, 3, 1, 0, 3, 1, 2, 1, 0, 2, 3, 1, 0, 3, 0, 1, 1, 3, 0, 0,
        2, 1, 0, 1, 0, 0, 2, 0, 1, 3, 0, 3, 1, 1, 1, 3], device='cuda:0')

### Lets now evaluate the test set as raw features for RNN model

In [19]:
labels_df = pd.DataFrame(columns=["file","label"])
for index in range(0,len(paths)):
    labels_df.loc[-1] = [os.path.basename(paths[index]),pred[index].cpu().numpy()]
    labels_df.index = labels_df.index + 1  # shifting index
    labels_df = labels_df.sort_index()
labels_df.to_csv("raw_rnn_test_lables.csv")